In [221]:
def face_detection(img, X_train_trans, pca, frame_row, frame_col):
    priority_queue = heapdict()
    img_row, img_col = img.shape
    for row in range(0, img_row-frame_row, 4):
        for col in range(0, img_col-frame_col, 4):
            frame = img[row:row+frame_row, col:col+frame_col]
            X_frame = np.ravel(frame)
            X_frame = X_frame[np.newaxis, :]
            X_frame_trans = pca.transform(X_frame)
            distances = np.linalg.norm(X_train_trans - X_frame_trans, axis=1)
            priority_queue[(row, col)] = np.min(distances)
    return priority_queue


In [223]:
def get_top_pixels(n_pixels, priority_queue):
    top_pixels = []
    for _ in range(n_pixels):
        row_col, _ = priority_queue.popitem()
        top_pixels.append(row_col)
    return top_pixels

In [272]:
def draw_dots(img, row_col_list, square_size):
    img_copy = np.copy(img)
    for row, col in row_col_list:
        img_copy[row:row+square_size, col:col+square_size] = 0
    return img_copy

### Используем изображение большой тройки

In [ ]:
big_3_img = img_as_float(imread('big_3_small.jpg'))
imshow(big_3_img)

Сравним с каким изображением похож Сталин и случайное изображение

In [ ]:
x = 90
y = 40
stalin = big_3_img[y:y+64, x:x+64]
compare_images(stalin, 0, 0, pca)

In [ ]:
compare_images(big_3_img, 10, 10, pca)

Видим, что лицо Сталина имеет меньше схожести чем случайный фрагмент фона, что плохо для алгоритма

Найдем 100 самых похожих на лицо области

In [ ]:
priority_queue = face_detection(big_3_img[:, :400], X_lfw_trans, pca, 64, 64)
top_pixels = get_top_pixels(4, priority_queue)

In [ ]:
imshow(draw_dots(big_3_img, top_pixels, 3))

## Используем фотографию beatles

In [ ]:
beatles_img = img_as_float(imread('Beatles.jpg', as_gray=True))
imshow(beatles_img)

Сравним результаты для лица и случайного фрагмента

In [ ]:
compare_images(beatles_img, 115, 115, pca)

In [ ]:
compare_images(beatles_img, 312, 328, pca)

Видим, что оценка лица гораздо ниже чем майки

In [ ]:
priority_queue = face_detection(beatles_img, X_lfw_trans, pca, 64, 64)
top_pixels = get_top_pixels(100, priority_queue)

In [ ]:
imshow(draw_dots(beatles_img, top_pixels, 5))

Видим, что наш алгортм не справился с задачей. Посмотрем какие оценки он дает частям, которые действительно отвечают за лицо